This notebook contains slides for presenting some functionality and implementation details of the Logtalk Jupyter kernel.

# A Jupyter Kernel for Logtalk

- Execute queries

In [ ]:
atom_length(atom,B).

- Define predicates 

In [ ]:
@user
my_append([], Res, Res).
my_append([H|T], List, [H|Res]) :-
  my_append(T, List, Res).

In [ ]:
my_append([1,2], [3,4], R).

- Provides the possibility of executing Logtalk code with Jupyter applications
- Replicates the standard Logtalk usage and adds convenience functionality


- **Execute queries**: Example showing additional functionality: `atom_length(atom, L)`
    - Exact name? *Tab* &rarr; completion
    - Argument order? *Shift+Tab* &rarr; inspection 
    - Note: **missing terminating full-stop**
        - Eliminate a cause for queries not to be run right away


- Also: **Define objects, protocols, categories, and predicates**


- Jupyter can be used to create notebooks consisting of cells like these ones
    - Source code and documentation
    - Create Assignments
    - Create slides for lectures like these ones
    - &rarr; Useful for teaching Logtalk

## Differentiating Term Types

- A code cell can contain a query or multiple terms to be interpreted as directives and clauses to be added to a file. In the later case, the first line must be one of `@user`, `@user+`, `@file FILENAME`,  or `@file+ FILENAME` (aka *cell magic*). The `+` variants append to an existing file instead of redefining it.

### Query

In [ ]:
X = [1,2,3], list::append(X, [4,5,6], Z).

### Predicate definition

In [ ]:
@user
fact(a).
fact(b).

In the next cell, the previous clauses are replaced by new ones:

In [ ]:
@user
fact(c).
fact(d).

But new clauses can be added instead by declaring the predicate *discontiguous*:

In [ ]:
@user
:- discontiguous(a/1).

In [ ]:
@user+
a(1).
a(2).

In [ ]:
@user+
b(3).

In [ ]:
@user+
a(4).

In [ ]:
listing(a/1), listing(b/1).

- A user might want to define a predicate in separate cells as illustrated above

### Object, protocol, and category definitions

Logtalk entities are preferably defined using the `@file FILENAME`, or `@file+ FILENAME` cell magic

In [ ]:
@file foo.lgt

:- object(foo).

    :- public(bar/0).
    bar :-
        write('Hello world!\n').

:- end_object.

In [ ]:
foo::bar.

## Handling Multiple Solutions

- Mimicking the usual backtracking mechanism

In [ ]:
list::member(M, [a,b,c]).

In [ ]:
jupyter::retry.

We can also simply type:

In [ ]:
retry.

- Problem of the Jupyter kernel: **user intraction** not supported

## Debugging

Load an example, compiling it in debug mode:

In [ ]:
set_logtalk_flag(debug, on), {elephants(loader)}.

Try one of examples queries:

In [ ]:
fred::number_of_legs(N).

Repeat the query printing the trace of a goal, starting by recompiling the example in debug mode:

In [ ]:
debugger::trace, fred::number_of_legs(N).

Turn off tracing and compiling in debug mode:

In [ ]:
debugger::notrace, set_logtalk_flag(debug, off).

- Debugging cannot be performed interactively
- Instead, print the trace of a goal

## Running Tests

In [ ]:
{ack(tester)}.

## Benchmarking Capabilities

- Whenever a query is executed, its runtime is stored in the database

In [ ]:
list::member(M, [1,2,3]).

In [ ]:
jupyter::print_query_time.

- Access the previous goal and its runtime

## Structured Output

- Display all possible results of a goal in a table

In [ ]:
@table
list::member(Member, [10,20,30,40]), Square is Member*Member.

## Printing terms

In [ ]:
@tree
a(1, b(2, c(3, 4))).

## Introspection

- Predicate inspection: *Shift + Tab*
    - Help retrieved `help/1`

- Various `jupyter` predicates
    - Access documentation with a help predicate

In [ ]:
jupyter::help.

- Various `juypter` (mostly convenience) predicates
- Difficult to remember all of them
    - In addition to completion and inspection: predicate to print all documentation

## Jupyter

- Originates from the **IPython** project
    - Enables interactive Python development
    - Several frontends, including a former version of **Jupyter Notebook**
        - Web application for handling Jupyter notebooks
        - Planned to be replaced by **JupyterLab**



- *Two-process model*:
<img style="float: right; max-width: 40%;" src="user_interaction_diagram.png">

    - Client process: responsible for user interaction
    - Kernel process: handles code execution

## Architecture

<img style="max-width: 80%;" src="architecture_diagram.png">






Kernel split in three:
- Extends IPython kernel: **inherits** the communication with a frontend via the ZeroMQ protocol


- Does not interpret Prolog itself
    - Starts an existing Prolog instance in a **subprocess**
        - Communicates with it according to the JSON-RPC 2.0 protocol
    - For any code execution **request**:
        - Sends a request message to the Prolog server containing the **code**
        - Prolog terms are read from the code and handled
    
    
- Make the kernel **extensible**: additional layer of a *kernel implementation* in between
    - **Responsible** for basically all functionality (e.g. handling Prolog **server**)
    - For every request the kernel receives, a **method** of the implementation class is called
    - Kernel started: loads **config** file
        - Can contain paths to interpreter-specific Python class files
    - By **extending** default implementation class and **overriding** methods
        - Kernel behaviour can be adjusted
    - Had to be done to support predicate inspection for both    
- Configure to start a different Prolog server

## Changing the Prolog Implementation

- Switch between Prolog backends on the fly
- The previous server process is kept running
    - When switching back, the database state has not changed

Several Prolog backends are supported and shortcuts are provided to switch to them if installed:

- LVM (`lvm`)
- SICStus Prolog (`sicstus`)
- SWI-Prolog (`swi`)
- Trealla Prolog (`trealla`)
- YAP (`yap`)

The above shortcuts assume Logtalk was installed using either one of the provided installers or by running the manual installation script (i.e. you can run e.g. Logtalk with SWI-Prolog by simply typing `swilgt`). But if you running Logtalk from a git clone directory, you will need to switch Prolog backends using the `jupyter::set_prolog_backend(BackendIntegrationScript)` predicate instead (e.g. `jupyter::set_prolog_backend(trealla)`).

In [ ]:
trealla.

Define our own list append predicate and consult it:

In [ ]:
@file app.pl
app([], List, List).
app([Head| Tail], List, [Head| Tail2]) :-
    app(Tail, List, Tail2).

In [ ]:
app([1,2], [3], R).

## Extending the Kernel

- At first, the kernel was developed for SICStus Prolog only
    - Adjusted for SWI-Prolog as well
    - Made extensible for further Prolog backends
    

- By **replacing the Prolog server**, the Python part can easily support a different implementation
    - Requirements: receive requests as JSON-RPC 2.0 messages, handle them, and send responses
    - Might be possible to further extend the existing server with conditional compilation
        - Advanced features might require significant changes

- By **overriding the `LogtalkKernelBaseImplementation` class**, most of the basic kernel behaviour can be adjusted

- Server replacement:
    - Portable code except for the stream redirection details that depend on the backend
    - Extend existing:
        - Implementing **basic code execution** should not require major effort
        - More **advanced features** might involve significant changes


- Replacement of the server does not suffice?: Python extension

## Future Work

- Support further Prolog backends
    - Or multiple versions of the same implementation

- Combine strengths of several Prolog backends
    - Kernel can be connected with multiple servers at once
    - Reusing results for another one should be relatively easy

- Send commands to all available Logtalk servers *at once*
    - Detect differences in the behaviour
    - Compare the performance by using the benchmarking functionality